In [1]:
# %load std_libraries.py
#Indicate operating environment and import core modules
import os 
location_input = input("what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = other")
location_dict = {'a': "C:\\Users\\BMH_work\\", 'b': "/home/heineike/",
                 'c': "C:\\Users\\Ben\\Documents\\", 'd':'you need to add your location to the location_dict'}
home_dir = location_dict[location_input]
print("home directory is " + home_dir)
base_dir = home_dir + os.path.normpath('github/y1000plus_tools') + os.sep
print("y1000plus_tools dir is " + base_dir )
y1000plus_dir_options = {'b':home_dir + os.path.normpath("genomes/y1000plus") + os.sep, 
                         'c': home_dir + os.path.normpath('github/expression_broad_data/expression_data/promoter_phylogenies/y1000plus') + os.sep
                        }
y1000plus_dir = y1000plus_dir_options[location_input]
print("y1000plus data dir is " + y1000plus_dir)

import sys

if not(base_dir in sys.path): 
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path" )

print("Importing y1000plus_tools.py")
from core import y1000plus_tools

io_library_path = home_dir + os.path.normpath('github/expression_broad_data/core')
if not(io_library_path in sys.path):
    sys.path.append(io_library_path)
    print("Added " + io_library_path + " to path" )

print("importing io_library.py")
import io_library
print('sys.path : \n')
print(sys.path)

%load_ext autoreload
%autoreload 2


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio import SeqIO
from ete3 import Tree, SeqMotifFace, TreeStyle, add_face_to_node, RectFace, NodeStyle  
#ete3 is not officially supported on windows, and so must be loaded via pip: 
# pip install -U https://github.com/etetoolkit/ete/archive/qt5.zip
# ref: https://groups.google.com/forum/#!topic/etetoolkit/6NblSBPij4o
#20181031: got this error message: twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.

# In order to view ete3 created trees on the gpucluster, you need to use a virtual X server:
from pyvirtualdisplay import Display
display = Display(visible=False, size=(1024, 768), color_depth=24)
display.start()


what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
home directory is /home/heineike/
y1000plus_tools dir is /home/heineike/github/y1000plus_tools/
y1000plus data dir is /home/heineike/genomes/y1000plus/
Added /home/heineike/github/y1000plus_tools/ to path
Importing y1000plus_tools.py
what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
home directory is /home/heineike/
y1000plus_tools dir is /home/heineike/github/y1000plus_tools/
y1000plus data dir is /home/heineike/genomes/y1000plus/
Added /home/heineike/github/expression_broad_data/core to path
importing io_library.py
Importing io_library.  If autoreload, may need to reset base_dir and data_processing dir 
  io_library.base_dir=base_dir 
 io_library.data_processing_dir = data_processing_dir
sys.path : 

['', '/home/lab/envs/seqanalysis/lib/python35.zip', '/home/lab/envs/seqanalysis/lib/python3.5', '/home/lab/envs/seqanalysis/lib/python3.5/plat-linux', '/home/

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1029'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1029'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [22]:
y1000_species = pd.read_csv(y1000plus_dir + "y1000_species_table.csv", index_col=0)

#Select only clades of interest: 
clades_of_interest = {'Saccharomycetaceae','Saccharomycodaceae','Phaffomycetaceae','CUG-Ser2','CUG-Ser1','Pichiaceae','CUG-Ala'}
#clades_of_interest = {'Saccharomycetaceae'}
y1000_species_subset = y1000_species[y1000_species['Major clade'].isin(clades_of_interest)]

outgroup = 'Yarrowia lipolytica'
#outgroup = 'Hanseniaspora vineae'  #For EGO2/4 - only Sacch so use different outgroup
y1000_species_subset = pd.concat([y1000_species_subset, y1000_species[y1000_species['Species name']==outgroup]])


In [23]:
#genes of interest: 

#NOtice how many of these genes are very close on the genome MPC3, SOL4, ENO1, GND2


gois = ['YGR043C','YLR354C', #Start strong Msn2/4 regulation NQM1/TAL1
       'YGR256W', 'YHR183W', #GND2/GND1 - Missing genes swapped between expression and stres.
        'YNR034W-A', 'YCR075W-A', #EGO4/EGO2 - lots of STREs 110_1183, 110_700
        'YMR105C', 'YKL127W', #PGM2/PGM1 - still has STRE but expression goes away
        #'YMR285C','YML118W', #NGL2/NGL3 - Poly-A RNA exonuclease - different OGs 
        'YGR254W','YHR174W', #ENO1/ENO2
        'YGR248W','YHR163W', #SOL4/SOL3 - strong PDS signal.  Not many orthologs for the duplicate - contradiction with orthogroups/YGOB somewhere
        'YGR243W','YHR162W', #MPC3/MPC2 - Start Weak/no Msn2/4 effect: 
        'YJL164C','YKL166C', #TPK1/TPK3   (but GIS1/RPH1 had an effect and there is a strong PDS) 
        'YDL021W','YOL056W', #GPM2/GPM3 - #Start average Msn2/4 effect: expression data missing ortholog for high, I'm missing ortholog from low 
        #'YPL230W','YMR182C' #USV1/RGM1 - Different OGs
        'YNL093W','YOR089C',  #YPT53/VPS21  110_5384, 110_5422 YPT53 is a singleton in this OG set
        'YOR237W','YPL145C'  # HES1/KES1
      ]




In [25]:
#Make promoter files for various genes of interest
#Load S.Cer lookup table: 
scer_lookup_fname = y1000plus_dir + os.path.normpath('id_lookups/saccharomyces_cerevisiae.csv')

scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
# target_y1000_id = scer_lookup.loc[goi,'y1000_id']

y1000_ids = dict(scer_lookup.loc[gois,'y1000_id'])
y1000_id_list = [y1000_ids[goi] for goi in gois]

goi_og_lookup, og_genes_lookup = y1000plus_tools.make_og_genes_lookup(y1000_id_list, y1000_species_subset)

more than one goi in same orthogroup :OG1193 found_genes: 
['110_4425', '110_757']
more than one goi in same orthogroup :OG1302 found_genes: 
['110_1562', '110_689']
more than one goi in same orthogroup :OG1407 found_genes: 
['110_5570', '110_2481']
more than one goi in same orthogroup :OG1488 found_genes: 
['110_326', '110_4266']
more than one goi in same orthogroup :OG1667 found_genes: 
['110_1753', '110_821']
more than one goi in same orthogroup :OG1859 found_genes: 
['110_3574', '110_2707']
more than one goi in same orthogroup :OG3991 found_genes: 
['110_4019', '110_1058']
more than one goi in same orthogroup :OG4796 found_genes: 
['110_1854', '110_2527']
more than one goi in same orthogroup :OG6005 found_genes: 
['110_1259', '110_2716']


In [42]:
# Checking for outgroup for EGO2/4
# y1000_species[y1000_species['Major clade']=='Sporopachydermia clade']['spec_og_id']
# genes_specs = [int(gene_ind.split('_')[0]) for gene_ind in ogcomb_genes]
# subset = y1000_species[y1000_species['spec_og_id'].isin(genes_specs)]
# subset[subset['Major clade']=='Saccharomycodaceae']


#EGO4/2 outgroup species is , protein is Hanseniaspora vineae '46_1782'

,original_genome_id,old_species_names,species_names_fig2,hex,id_table S1,Species name,Major clade,clade_order,Family,Genus,CUG usage,spec_og_id
ID,,,,,,,,,,,,
306,hanseniaspora_valbyensis,Hanseniaspora_valbyensis,Hanseniaspora_valbyensis,#12958A,255,Hanseniaspora valbyensis,Saccharomycodaceae,11,Saccharomycodaceae,Hanseniaspora,Leu,45
307,hanseniaspora_uvarum,Hanseniaspora_uvarum,Hanseniaspora_uvarum,#12958A,254,Hanseniaspora uvarum,Saccharomycodaceae,11,Saccharomycodaceae,Hanseniaspora,Leu,44
308,yHMPu5000035695_hanseniaspora_pseudoguilliermo...,yHMPu5000035695_Hanseniaspora_pseudoguilliermo...,Hanseniaspora_pseudoguilliermondii,#12958A,258,Hanseniaspora pseudoguilliermondii,Saccharomycodaceae,11,Saccharomycodaceae,Hanseniaspora,Leu,323
309,yHMPu5000035684_kloeckera_hatyaiensis_160519,yHMPu5000035684_Kloeckera_hatyaiensis,Kloeckera_hatyaiensis,#12958A,259,Kloeckera hatyaiensis,Saccharomycodaceae,11,Saccharomycodaceae,Kloeckera,Leu,316
311,hanseniaspora_vinae,Hanseniaspora_vinae,Hanseniaspora_vineae,#12958A,256,Hanseniaspora vineae,Saccharomycodaceae,11,Saccharomycodaceae,Hanseniaspora,Leu,46
330,yHMPu5000034957_hanseniaspora_osmophila_160519,yHMPu5000034957_Hanseniaspora_osmophila,Hanseniaspora_osmophila,#12958A,261,Hanseniaspora osmophila,Saccharomycodaceae,11,Saccharomycodaceae,Hanseniaspora,Leu,247
331,yHMPu5000035696_hanseniaspora_singularis_160519,yHMPu5000035696_Hanseniaspora_singularis,Hanseniaspora_singularis,#12958A,257,Hanseniaspora singularis,Saccharomycodaceae,11,Saccharomycodaceae,Hanseniaspora,Leu,324
332,yHMPu5000034963_hanseniaspora_clermontiae_160519,yHMPu5000034963_Hanseniaspora_clermontiae,Hanseniaspora_clermontiae,#12958A,260,Hanseniaspora clermontiae,Saccharomycodaceae,11,Saccharomycodaceae,Hanseniaspora,Leu,248


In [46]:
#Combine two orthogroups because we know they are for paralogs (in this case EGO4 and EGO2)

##Should make a function for this that starts with og numbers and uses og_genes_lookup. 

# def combine_OGs(og1, og2): 
#     for 
# #EGO2: 110_700
# og1_line = 'OG6958: 299_59 145_2563 54_1183 57_473 61_2336 65_3281 135_4298 166_2391 223_835 300_4167 109_2203 110_700 111_1752 112_2029 113_3346 114_2951 115_2909 133_1437 136_2529 207_1375 208_1837 217_1311 220_5130 48_1691 50_4079 51_1157 56_2944 58_3655 59_3473 62_3438 63_1065 64_717 219_4548 222_670 164_5034 19_2260 218_1421 341_2448 342_2639 95_2057 98_1079 38_55 96_2445 39_3944 40_1411 46_1782 60_1097 7_3017 94_766 247_4091 41_4060 55_1119 155_5785 97_4093 152_1088 248_2694 316_3531 323_2667 324_593 44_2693 45_300'
# og1_name = og1_line.split(':')[0]
# og1_genes = og1_line.split(':')[1].split(' ')[1:]

# #EGO4: 110_1183
# og2_line = 'OG10444: 100_245 100_526 145_324 101_2474 151_3036 155_4004 48_4329 109_2086 110_1183 111_3614 112_552 113_3707 114_2331 115_2979 19_4114 94_4005 95_3420 96_4426 97_67 98_3762'
# og2_name = og2_line.split(':')[0]
# og2_genes = og2_line.split(':')[1].split(' ')[1:]

# ogcomb_name = og1_name + '_' + og2_name
# ogcomb_genes = og1_genes + og2_genes

# y1000_species_subset_ogs = set(y1000_species_subset['spec_og_id'])
# genes_specs = {gene_ind for gene_ind in ogcomb_genes if int(gene_ind.split('_')[0]) in y1000_species_subset_ogs}

# goi_og_lookup['110_700'] = ogcomb_name
# goi_og_lookup['110_1183'] = ogcomb_name

# og_genes_lookup[ogcomb_name] = genes_specs


In [26]:
#Test to see which orthogroups have something from outgroup
for y1000_id, goi in y1000_ids.items():
    print(y1000_id)
    og_genes = og_genes_lookup[goi_og_lookup[goi]]
    for gene in og_genes: 
        if gene.split('_')[0]=='144':
            print(gene)

#Results: 
#GPM2/GPM3 no ortholog in Y. Lipolitica
#TPK1/TPK3 no ortholog in Y. lipolitica (interesting...)  
# Need to find new outgroup for those

YKL127W
144_1600
YGR243W
144_4356
YHR163W
144_4616
YNR034W-A
YCR075W-A
YHR174W
144_3401
YOR089C
144_901
YGR254W
144_3401
YHR162W
144_4356
YJL164C
YLR354C
144_4100
YOR237W
144_5749
YGR256W
144_1480
YHR183W
144_1480
YDL021W
YOL056W
YNL093W
YGR043C
144_4100
YMR105C
144_1600
YGR248W
144_4616
YKL166C
YPL145C
144_5749


In [27]:
#Consolidate pairs with orthogroups
goi_pair_og_lookup =  {'NQM1_TAL1': 'OG1667', 
                     'GND2_GND1': 'OG1488', 
                     'PGM2_PGM1': 'OG1302', 
                     'ENO1_ENO2': 'OG1193', 
                     'SOL4_SOL3': 'OG3991', 
                     'MPC3_MPC2': 'OG1859', 
                     'TPK1_TPK3': 'OG4796',
                     'GPM2_GPM3': 'OG6005',
                     'EGO4_EGO2': 'OG6958_OG10444', 
                     'HES1_KES1': 'OG1407'
                    }

In [28]:
#For a given goi_pair, get y1000 promoters and proteins and deposit into a file
L_prom = 1000

goi_pair = 'NQM1_TAL1' #'ENO1_ENO2'
print(goi_pair)
og = goi_pair_og_lookup[goi_pair]
og_genes =  og_genes_lookup[og]

#For a given orthogroup extract promoters

y1000plus_tools.extract_promoters(L_prom, og, og_genes, y1000_species_subset, fname_string=goi_pair)
print(goi_pair + ' promoter extraction complete')

protein_fname = goi_pair + '_' + og
y1000plus_tools.extract_promoter_seqs(og_genes, protein_fname, y1000_species_subset)
print(goi_pair + ' protein sequence extraction complete')

NQM1_TAL1
promoter region extends past the scaffold, genome_name = yHMPu5000034952_citeromyces_siamensis_160519 Gene = Seq_5207, L_prom = 1000
scaffold ends at the end of the gene, genome_name = yHMPu5000034952_citeromyces_siamensis_160519 Gene = Seq_5207
promoter region extends past the scaffold, genome_name = yHMPu5000034963_hanseniaspora_clermontiae_160519 Gene = Seq_729, L_prom = 1000
promoter region extends past the scaffold, genome_name = yHMPu5000034865_zygosaccharomyces_kombuchaensis_160519 Gene = Seq_3034, L_prom = 1000
NQM1_TAL1 promoter extraction complete
yHMPu5000035252_yamadazyma_nakazawae_160928
yHMPu5000034866_zygosaccharomyces_bisporus_160519
metschnikowia_santaceciliae
yHMPu5000034624_pichia_nakasei_160519
yHAB148_kazachstania_intestinalis_160519
yHMPu5000034632_candida_athensensis_160519
babjeviella_inositovora
yHMPu5000034643_candida_schatavii_160519
nakaseomyces_delphensis
yHAB166_kazachstania_yakushimaensis_160519
lachancea_fermentati
priceomyces_haplophilus
nakas

yHMPu5000035671_phaffomyces_antillensis_160613
lachancea_lanzarotensis
hyphopichia_burtonii
yHMPu5000035268_wickerhamomyces_hampshirensis_160928
yHMPu5000041833_candida_tammaniensis_160928
yHMPu5000041863_candida_gorgasii_160928
yHMPu5000034622_pichia_occidentalis_160519
yHMPu5000035681_candida_gotoi_160519
ascoidea_asiatica
yHMPu5000034865_zygosaccharomyces_kombuchaensis_160519
yHMPu5000035048_barnettozyma_salicaria_160519
yHMPu5000034597_candida_stellimalicola_160519
metschnikowia_aberdeeniae
yHMPu5000034615_saturnispora_zaruensis_160519
yHAB153_kazachstania_rosinii_160519
yHMPu5000034876_tetrapisispora_iriomotensis_160519
spathaspora_gorwiae
yHMPu5000035022_candida_emberorum_160519
yHMPu5000035687_cyberlindnera_suaveolens_160613
scheffersomyces_stipitis
metschnikowia_drakensbergensis
yHMPu5000035667_kurtzmaniella_cleridarum_160928
metschnikowia_ipomoeae
yHMPu5000034891_ogataea_pini_160519
yHMPu5000034988_candida_fructus_160519
metschnikowia_lockheadii
eremothecium_coryli
candida_aur

In [87]:
## Did this prior to adding in outgroup
# #Consolidate pairs with orthogroups
# goi_pair_og_lookup =  {'NQM1_TAL1': 'OG1667', 
#                      'GND2_GND1': 'OG1488', 
#                      'PGM2_PGM1': 'OG1302', 
#                      'ENO1_ENO2': 'OG1193', 
#                      'SOL4_SOL3': 'OG3991', 
#                      'MPC3_MPC2': 'OG1859', 
#                      'TPK1_TPK3': 'OG4796',
#                      'GPM2_GPM3': 'OG6005'
#                     }


# L_prom = 1000

# for goi_pair, og in goi_pair_og_lookup.items(): 
#     print(goi_pair)
#     og_genes =  og_genes_lookup[og]
    
#     #For a given orthogroup extract promoters

#     y1000plus_tools.extract_promoters(L_prom, og, og_genes, y1000_species_subset, fname_string=goi_pair)
#     print(goi_pair + ' promoter extraction complete')
    
#     protein_fname = goi_pair + '_' + og
#     y1000plus_tools.extract_promoter_seqs(og_genes, protein_fname, y1000_species_subset)
#     print(goi_pair + ' protein sequence extraction complete')

In [29]:


# Mafft command 

#Make tree: 

#Use Mafft to construct an MSA

goi_pair = 'NQM1_TAL1'#'ENO1_ENO2'
og = goi_pair_og_lookup[goi_pair]

protein_fname_base = y1000plus_dir + os.path.normpath('proteins_og/' + goi_pair + '_' + og)


#Command: 
mafft_command = ['mafft', '--genafpair', '--maxiterate', '1000', 
                protein_fname_base + '.fasta', 
                '>',
                protein_fname_base + '_aln.fasta']

#Running it from this notebook seems slow
#!{" ".join(mafft_command)}

print(" ".join(mafft_command))

#This is the E-ENS-i algorithm which is suitable for conserved regions separated by long unaligned gaps. https://mafft.cbrc.jp/alignment/software/algorithms/algorithms.html
#tree command: (designated outgroup)

outgroup_gene = None
prot_seqs = SeqIO.parse(protein_fname_base + '.fasta', "fasta")
for seq in prot_seqs: 
    if seq.id.split('Seq')[0]=='yarrowia_lipolytica_':
    #if seq.id.split('Seq')[0]=='hanseniaspora_vinae_':
        outgroup_gene = seq.id
        
if outgroup_gene == None: 
    print('No yarrowia_lipolytica ortholog - need to pick a different outgroup')

#Note: for iqtree to run you need to activate the seqanalysis python environment: 
#. activate /home/lab/envs/seqanalysis

iqtree_command = ["iqtree", 
                  "-s" , protein_fname_base + '_aln_trimmed.fasta',
                  "-nt", "AUTO",  #automatically determines number of threads 
                  "-o", outgroup_gene]
print(" ".join(iqtree_command))

#Current trimming protocol - open up on Jalview and trim by eye. Save as _trimmed.fasta, then find and replace
# all "/" at the end of each gene name with a space. 


# #Use the MSA to build a tree

# #from that tree for all post WGD species, 
#     #see if gene is closer to "low" or "high" goi.  Assign pre_wgh, low, or high label
#     #could verify that outgroup doesn't appear between target and S.Cer low or high gene as well. 
    
# #Order search results based on the tree


mafft --genafpair --maxiterate 1000 /home/heineike/genomes/y1000plus/proteins_og/NQM1_TAL1_OG1667.fasta > /home/heineike/genomes/y1000plus/proteins_og/NQM1_TAL1_OG1667_aln.fasta
iqtree -s /home/heineike/genomes/y1000plus/proteins_og/NQM1_TAL1_OG1667_aln_trimmed.fasta -nt AUTO -o yarrowia_lipolytica_Seq_4101


In [61]:
#Find STRE/PDS/TATA box in promoters

# Scan motifs 
output_format = 'full'
motif_dict = {'STRE': 'CCCCT', 'TATA': 'TATA[AT]A[AT][AG]', 'PDS': 'AGGGAT'}   
sequence_context = 2
L_prom = 1000


goi_pair ='ENO1_ENO2'
og = goi_pair_og_lookup[goi_pair]

promoters_fname = y1000plus_dir + os.path.normpath('promoter_sets/' + og + '_' + goi_pair + '.fasta')

#promoters_fnames = os.listdir(data_processing_dir + os.path.normpath('promoter_phylogeny/promoter_sets') )

#og_promoter_fnames = { fname.split('_')[0] : fname for fname in promoters_fnames }

#promoters_fname = data_processing_dir + os.path.normpath('promoter_phylogeny/promoter_sets/') + os.sep + og_promoter_fnames[og]

proms = io_library.exact_promoter_scan_from_fasta(promoters_fname, motif_dict, output_format = output_format, sequence_context = sequence_context, seq_key_func=y1000plus_tools.seq_key_func, L_prom = L_prom)

proms['spec_genome_name'] = [item.split('@')[0] for item in proms.index]

proms.index.name = 'seq_id'
proms.reset_index(inplace=True)
merge_columns = y1000_species_subset.loc[:,['original_genome_id', 'species_names_fig2']].rename(columns={'original_genome_id':'spec_genome_name'})
proms = proms.merge(merge_columns, how = 'left', on = 'spec_genome_name')
proms.set_index('seq_id', inplace=True)
                                                   
# sort_dict = {spec_name: jj for jj, spec_name in enumerate(tree_order_plus_extra)}
# sort_column = [sort_dict[spec_name] for spec_name in proms['species_names_fig2']]
# proms['sort_column'] = sort_column
# proms_sorted = proms.sort_values(by='sort_column')


# fig, ax = plt.subplots(figsize = (20,5)) 

# ax.bar(x = range(0,len(proms_sorted)), height=proms_sorted.TATA_count, tick_label = proms_sorted.species_names_fig2)
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# fig.tight_layout()

In [78]:
#Visualize the tree with sequence next to it

# goi_pair_og_lookup =  {'NQM1_TAL1': 'OG1667', 
#                      'GND2_GND1': 'OG1488', 
#                      'PGM2_PGM1': 'OG1302', 
#                      'ENO1_ENO2': 'OG1193', 
#                      'SOL4_SOL3': 'OG3991', 
#                      'MPC3_MPC2': 'OG1859', 
#                      'TPK1_TPK3': 'OG4796',
#                      'GPM2_GPM3': 'OG6005'
#                     }

sacc_families = {'Candida': 'Post_WGH',
                 'Kazachstania': 'Post_WGH',
                 'Nakaseomyces': 'Post_WGH',
                 'Naumovozyma': 'Post_WGH',
                 'Saccharomyces': 'Post_WGH',
                 'Tetrapisispora': 'Post_WGH',
                 'Vanderwaltozyma': 'Post_WGH',
                 'Yueomyces': 'Post_WGH',
                 'Zygosaccharomyces': 'ZT',
                 'Zygotorulaspora': 'ZT',
                 'Torulaspora': 'ZT',
                 'Kluyveromyces': 'KLE',
                 'Lachancea': 'KLE',
                 'Eremothecium': 'KLE',
                 'Ashbya': 'KLE'
                }


less_nonsacc=True

#Color Node by species: 
sacc_colors = {'KLE': "#C6AFE9", 
               'ZT': "YellowGreen",
               'Post_WGH': "LightYellow"
              }

# maj_clade_colors = {'Saccharomycodaceae': "BlanchedAlmond",
#     'Phaffomycetaceae': "Bisque",
#     'CUG-Ser2': 'NavajoWhite',
#     'CUG-Ser1': "Wheat",
#     'Pichiaceae': "BurlyWood",
#     'CUG-Ala':"Tan"
#     }

pre_wgd_purple = "#bb95d6"
maj_clade_colors = {'Saccharomycodaceae': pre_wgd_purple,
    'Phaffomycetaceae': pre_wgd_purple,
    'CUG-Ser2': pre_wgd_purple,
    'CUG-Ser1': pre_wgd_purple,
    'Pichiaceae': pre_wgd_purple,
    'CUG-Ala': pre_wgd_purple
    }

goi_pair = 'ENO1_ENO2'
og = goi_pair_og_lookup[goi_pair]

protein_fname_base = y1000plus_dir + os.path.normpath('proteins_og/' + goi_pair + '_' + og)
tree_fname = protein_fname_base + '_aln_trimmed.fasta.treefile'
t = Tree(tree_fname)
t.ladderize()
L_prom = 700
height = 15
seq = '-'*L_prom

motif_colors = {'PDS': 'yellow', 'TATA': 'blue', 'STRE': 'red'}
motif_lengths = {'PDS': 12, 'TATA': 16, 'STRE': 10 }  #They are double the size

#box params:
width_box = 20
height_box = 30

cmap_STRE = cm.get_cmap('Reds')

vmin = 0.0
vmax = 8.0

norm = colors.Normalize(vmin=vmin, vmax=vmax)

cmap_TATA = cm.get_cmap("Blues")

if less_nonsacc: 
    nonsacc_visualization_subset = pd.read_csv(y1000plus_dir + 'species_visualization_subset.csv')
    species_subset = ( set(nonsacc_visualization_subset['original_genome_id']) | \
                       set(y1000_species_subset[y1000_species_subset['Major clade']=='Saccharomycetaceae']['original_genome_id']) | \
                       set(y1000_species[y1000_species['Species name']==outgroup]['original_genome_id'])
                     )
    node_subset = []
    #For each node in the tree:
    for node in t.iter_leaves():  
        #Get the promoter sequence with motif info, make it into a motif list
        species = '_'.join(node.name.split('_')[:-2])
        if species in species_subset: 
            node_subset.append(node.name)

    t.prune(node_subset)
    t.ladderize()


#For each node in the tree:
for node in t.iter_leaves():  
    #Get the promoter sequence with motif info, make it into a motif list
    species = '_'.join(node.name.split('_')[:-2])
    gene_id = species + '@' + '_'.join(node.name.split('_')[-2:])
    
    #color node by major clade / family if in Sacch clade
    species = '_'.join(node.name.split('_')[:-2])
    row = y1000_species_subset[y1000_species_subset['original_genome_id']==species]
    maj_clade = row['Major clade'].values[0]

    if maj_clade == 'Saccharomycetaceae':
        genus = row['Genus'].values[0]
        node_color = sacc_colors[sacc_families[genus]]
    elif species == 'yarrowia_lipolytica':
    #elif species == 'hanseniaspora_vinae':
        node_color = 'LightGrey'
    else: 
        node_color = maj_clade_colors[maj_clade]
    
    nstyle = NodeStyle()
    nstyle['bgcolor']=node_color
    node.set_style(nstyle)
    
    prom_results = proms.loc[gene_id]
    
#     simple_motifs = [
#             # seq.start, seq.end, shape, width, height, fgcolor, bgcolor
#             [10, 60, ">", None, 10, "black", "red", None],
#             [120, 150, "<", None, 10, "black", "red", None]
#     ]
    motifs = []
    
    for motif_name in ['TATA', 'STRE']: #, 'PDS']:   #Leaving out PDS
        motif_len = motif_lengths[motif_name]
        motif_color = motif_colors[motif_name]
        if prom_results[motif_name + '_count'] >0:

            for motif in prom_results[motif_name + '_full_features']:
                loc = motif[0]
                if loc <= L_prom:
                    direction = motif[1]
                    shape = '>'
                    start = L_prom-loc
                    stop = L_prom-loc + motif_len
                    if direction == 'rev':
                        shape = '<'
                        start = L_prom-loc-motif_len
                        stop = L_prom -loc
                    motifs.append([start,stop,shape,None, height, "black", motif_color, None])

    seqFace = SeqMotifFace(seq, motifs=motifs, seq_format="-")
    node.add_face(seqFace, column=0, position="aligned")
    
    
    #Add face for STRE count within 500bp
    
    motif_name = 'STRE'
    L_STRE_count = 500
    
    N_STRE = 0
    for result in prom_results['STRE_full_features']:
        if result[0]<L_STRE_count:
            N_STRE = N_STRE+1
     
    rgb = colors.to_hex(cmap_STRE(norm(N_STRE)))
    
    rectFace_STRE = RectFace(width=width_box, height=height_box, fgcolor='black', bgcolor=rgb, 
                        label= {"text": str(N_STRE), 
                                "color": 'black', 
                                "fontsize": 10, 
                                "font": 'Verdana'
                               }
                       )
    node.add_face(rectFace_STRE, column = 1, position= "aligned")
    
    motif_name = 'TATA'
    L_TATA = 300
    
    TATA = 0.0
    for result in prom_results['TATA_full_features']:
        if result[0]<L_TATA:
            TATA = 1.0
     
    rgb = colors.to_hex(cmap_TATA(TATA))
    
    rectFace_TATA = RectFace(width=width_box, height=height_box, fgcolor='black', bgcolor=rgb)
    
    node.add_face(rectFace_TATA, column=2, position="aligned")
    

#t.render('%%inline')
fname = home_dir + 'scratch/' + goi_pair + '.svg'
t.render(fname)


{'faces': [[201.08507860725913,
   2528.0,
   451.0850786072591,
   2544.0,
   168,
   'nakaseomyces_bacillisporus_Seq_4314'],
  [710.6245635178914, 2535.5, 761.6245635178914, 2536.5, 168, ''],
  [760.6245635178914,
   2528.0868863285964,
   772.4062319302121,
   2543.913113671404,
   168,
   ''],
  [771.6245635178914, 2535.5, 1236.6245635178914, 2536.5, 168, ''],
  [1235.9227434146032,
   2528.0425410992134,
   1253.6245635178914,
   2543.957458900786,
   168,
   ''],
  [1252.6245635178914, 2535.5, 1262.6245635178914, 2536.5, 168, ''],
  [1261.6245635178914,
   2528.0425410992134,
   1279.3263836211795,
   2543.957458900786,
   168,
   ''],
  [1278.6245635178914, 2535.5, 1411.6245635178914, 2536.5, 168, ''],
  [1416.6245635178914,
   2520.5,
   1437.6245635178914,
   2551.5,
   168,
   {'color': 'black', 'font': 'Verdana', 'fontsize': 10, 'text': '0'}],
  [1436.6245635178914, 2520.5, 1457.6245635178914, 2551.5, 168, None],
  [215.9564193947554,
   1058.0,
   630.9564193947554,
   1074